In [73]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import sys
import os
import time

In [78]:
def main():
    
    name = 'bbc.wav'
    sound = AudioSegment.from_mp3("bbc.mp3")
    sound.export(name, format="wav")
    sound = AudioSegment.from_wav(name)
    
    
    silence_thresh=-30     # 小于-70dBFS以下的为静默
    min_silence_len=400    # 静默超过700毫秒则拆分
    length_limit=500    # 拆分后每段不得超过5s
    abandon_chunk_len=500   # 放弃小于500毫秒的段
    joint_silence_len=100  # 段拼接时加入100毫秒间隔用于断句
    split(sound,silence_thresh,min_silence_len,length_limit,abandon_chunk_len,joint_silence_len)

In [79]:
def split(chunk,silence_thresh,min_silence_len,length_limit,abandon_chunk_len,joint_silence_len):
    print('开始拆分(如果录音较长，请耐心等待)\n',' *'*30)
    name = 'bbc.wav'
    chunks = split_on_silence(\
        chunk,min_silence_len=min_silence_len,silence_thresh=silence_thresh)
    
    print('拆分结束，返回段数:',len(chunks),'\n',' *'*30)
    # 放弃长度小于0.5秒的录音片段
    for i in list(range(len(chunks)))[::-1]:
        if len(chunks[i])<=abandon_chunk_len:
            chunks.pop(i)
    print('取有效分段：',len(chunks))

    # 时间过短的相邻段合并，单段不超过1分钟
    chunks = chunk_join_length_limit(chunks,joint_silence_len=joint_silence_len,length_limit=length_limit)
    print('合并后段数：',len(chunks))

    # 保存前处理一下路径文件名
    if not os.path.exists('./chunks'):os.mkdir('./chunks')
    namef,namec = os.path.splitext(name)
    namec = namec[1:]

    # 保存所有分段
    total = len(chunks)
    for i in range(total):
        new = chunks[i]
        save_name = '%s_%04d.%s'%(namef,i,namec)
        new.export('./chunks/'+save_name, format=namec)
        print('%04d'%i,len(new))
    print('保存完毕')


In [80]:
def chunk_join_length_limit(chunks,joint_silence_len,length_limit):
    '''
    将声音文件合并，并限定单句最长时间，返回结果为列表形式
    Args:
        chunk: 录音文件
        joint_silence_len: 合并时文件间隔
        length_limit：合并后单个文件长度不超过该值
        adjust_chunks：合并后的列表
    '''
    # 
    silence = AudioSegment.silent(duration=joint_silence_len)
    adjust_chunks=[]
    temp = AudioSegment.empty()
    for chunk in chunks:
        print(adjust_chunks)
        length = len(temp)+len(silence)+len(chunk) 
        
        if length<5000:
            temp+=silence+chunk
        else: 
            adjust_chunks.append(temp)
            temp=chunk
    else:
        adjust_chunks.append(temp)
        
    return adjust_chunks



main(0

In [81]:
main()

开始拆分(如果录音较长，请耐心等待)
  * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0257 4897
0258 2886
0259 4580
0260 3535
0261 4605
0262 3961
0263 4407
0264 4521
0265 3836
0266 2155
0267 4130
0268 5484
0269 3845
0270 2958
0271 6075
0272 3706
0273 5755
0274 2756
0275 3258
0276 5813
0277 3628
0278 4510
0279 4499
0280 2790
0281 5054
0282 4022
0283 3889
0284 6555
0285 3232
0286 3136
0287 10747
0288 4975
0289 5250
保存完毕
